<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Итак для начала загрузим все что нам нужно из библиотек. Посмотрим данные разными способами, при необходимости изменим типы и обработаем пропустки. Разделим на обучающую и валидационную и тестовую для итоговой проверки.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
data = pd.read_csv('Churn.csv')
#на сервере:
#data = pd.read_csv('/datasets/Churn.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [5]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата

- Exited - целевой признак!!! 

Первое что бросилось в глаза - Tenure. Несколька незаполненных строк. Я думаю, что замена средним будет хорошим решением.

In [6]:
data['Tenure'] = data['Tenure'].fillna(value = data['Tenure'].mean())

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Далее: в признаках есть такие, что не влияют на центральный: RowNumber, CustomerId, Surname. Для лучшей работы модели - просто исключим их.

In [8]:
data = data.drop(['RowNumber','CustomerId','Surname'], axis = 1)

In [9]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


Два признака Gender и Geography имеют строковый тип данных, это нужно исправить техникой прямого кодирования.

In [10]:
data = pd.get_dummies(data, columns = ['Geography', 'Gender'], drop_first = True)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Age                10000 non-null  int64  
 2   Tenure             10000 non-null  float64
 3   Balance            10000 non-null  float64
 4   NumOfProducts      10000 non-null  int64  
 5   HasCrCard          10000 non-null  int64  
 6   IsActiveMember     10000 non-null  int64  
 7   EstimatedSalary    10000 non-null  float64
 8   Exited             10000 non-null  int64  
 9   Geography_Germany  10000 non-null  uint8  
 10  Geography_Spain    10000 non-null  uint8  
 11  Gender_Male        10000 non-null  uint8  
dtypes: float64(3), int64(6), uint8(3)
memory usage: 732.5 KB


Масштабирование признаков. 

In [12]:
data.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
count,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,4.99769,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700,0.250900,0.247700,0.545700
std,96.653299,10.487806,2.76001,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769,0.433553,0.431698,0.497932
min,350.000000,18.000000,0.00000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000,0.000000,0.000000
25%,584.000000,32.000000,3.00000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,0.000000,0.000000,0.000000
50%,652.000000,37.000000,4.99769,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,0.000000,0.000000,1.000000
75%,718.000000,44.000000,7.00000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,1.000000,0.000000,1.000000
max,850.000000,92.000000,10.00000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,1.000000,1.000000,1.000000


In [13]:
numeric = ['Age','Balance','EstimatedSalary', 'CreditScore']

In [14]:
scaler = StandardScaler()

выделим целевой признак и разделим данные на обучающую, валидационную, тестовые выборки 

In [15]:
data_features = data.drop(['Exited'], axis = 1)
data_target = data['Exited']

In [16]:
features_train,features_test, target_train, target_test = train_test_split(data_features, data_target, test_size = .4,
                                                                           stratify = data_target, random_state = 12345)
features_test,features_valid, target_test, target_valid = train_test_split(features_test, target_test, test_size = 0.5,
                                                                            stratify = target_test, random_state = 12345)

In [17]:
scaler.fit(features_train[numeric])

StandardScaler()

In [18]:
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

In [19]:
features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
2837,-1.040434,0.953312,6.00000,0.774657,1,0,1,-0.119110,1,0,0
9925,0.454006,-0.095244,4.99769,1.910540,1,1,1,-0.258658,0,0,0
8746,0.103585,-0.476537,8.00000,0.481608,2,0,1,1.422836,0,0,1
660,-0.184996,0.190726,4.99769,0.088439,1,1,1,-1.160427,1,0,0
3610,-0.720933,1.620574,1.00000,0.879129,1,1,0,0.113236,0,0,0


## Исследование задачи

Исследуем баланс классов.

In [20]:
target_train.value_counts(normalize = True)

0    0.796333
1    0.203667
Name: Exited, dtype: float64

В наших данных явно больше 0. Сначала будем обучать модель без учета дисбаланса классов.

Будем обучать виды моделей для получения лучшего результата: дерево, случайный лес, логистическую регрессию.

Лес. Будем искать лучшую модель с разными гиперпараметрами: лубину, min_samples_split, min_samples_leaf.

In [21]:
n_estimators =  [int(x) for x in range(10, 50,10)]
depth = [int(x) for x in range(3, 8)]
min_samples_split = [2,3,4,5,6]
min_samples_leaf = [1,2,3,4]

In [22]:
rf_grif = {
    'n_estimators':n_estimators,
    'max_depth':depth,
    'min_samples_split' : min_samples_split,
    'min_samples_leaf': min_samples_leaf
}
model_forest_unb = GridSearchCV( estimator = RandomForestClassifier(),scoring = 'f1' ,param_grid = rf_grif, cv = 5)

In [23]:
model_forest_unb.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'min_samples_leaf': [1, 2, 3, 4],
                         'min_samples_split': [2, 3, 4, 5, 6],
                         'n_estimators': [10, 20, 30, 40]},
             scoring='f1')

In [24]:
predictions_forest_unb = model_forest_unb.predict(features_valid)
probabilities_forest_unb = model_forest_unb.predict_proba(features_valid)

In [25]:
print('Модель леса без учета балансов классов:')
print('F1 = ',f1_score(target_valid, predictions_forest_unb))
print('AUC ROC', roc_auc_score(target_valid, probabilities_forest_unb[:,1]))

Модель леса без учета балансов классов:
F1 =  0.5187601957585645
AUC ROC 0.852988581802141


Модель леса показала метрику f1 0,52 а auc_roc 0.85

Дерево. Гиперпараметры буду менять вручную.

In [26]:
f_one_best = 0
for min_samples_leaf in range (1,7):
    for min_samples_split in range(2,12):
        for depth in range(2, 12): 
            model_tree = DecisionTreeClassifier(max_depth = depth, random_state=12345, 
                                                min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf)
            model_tree.fit(features_train, target_train)
            predictions = model_tree.predict(features_valid)
            f_one = f1_score(target_valid,predictions)
            if f_one > f_one_best:
                model_tree_best = model_tree
                f_one_best = f_one 
print(f_one_best)            

0.5590433482810164


In [27]:
probabilities_tree_unb = model_tree.predict_proba(features_valid)
predictions_tree_unb = model_tree_best.predict(features_valid)
#print(,accuracy_score(target_valid, predictions))
print('F1 = ',f_one_best)  
print('AUC ROC =', roc_auc_score(target_valid, probabilities_tree_unb[:,1]))

F1 =  0.5590433482810164
AUC ROC = 0.7734082310353496


Тут метрика f1 равна 0,559(все равно меньше чем нужно), AUC ROC 0.77/

Логистическая регрессия.

In [28]:
model_log_unb = LogisticRegression(random_state=12345) # class_weight='balanced')
model_log_unb.fit(features_train, target_train)
predictions_log = model_log_unb.predict(features_valid)
probabilities_log_unb = model_log_unb.predict_proba(features_valid)
f_one_log = f1_score(target_valid,predictions)
print('F1 = ',f_one)
print('AUC ROC =', roc_auc_score(target_valid, probabilities_log_unb[:,1]))

F1 =  0.5150214592274678
AUC ROC = 0.7505209369616149


Логистическая модель сработала быстрее всех. F1 = 0,51 а AUC ROC = 0.75

Чет все плохо совсем. Ни одна модель не справилась с задачей f1 больше 0,59 на валидационной выборке. Лучше всех справился Лес, хуже всех. 

## Борьба с дисбалансом

Напомним, что до этого мы исследовали модели без учета дисбаланса классов. Проверим насколько все серьезно.

In [29]:
target_train.value_counts(normalize = True)

0    0.796333
1    0.203667
Name: Exited, dtype: float64

Единиц меньше в 4 раза. Разделим обучающую выборку и ответы на отдельные по классу ответа. Установим переменную repeat 4.

In [30]:
features_zeros_train = features_train[target_train == 0]
features_ones_train = features_train[target_train == 1]
target_zeros_train = target_train[target_train == 0]
target_ones_train = target_train[target_train == 1]

In [31]:
repeat = 4

Увеличим единицы в четыре раза и соединим обратно обучающую выборку и ответы.

In [32]:
features_upsampled = pd.concat([features_zeros_train] + [features_ones_train] * repeat)
target_upsampled = pd.concat([target_zeros_train] + [target_ones_train] * repeat)

Перемешаем и проверим что получилось.

In [33]:
features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

In [34]:
target_upsampled.value_counts(normalize = True)

1    0.50569
0    0.49431
Name: Exited, dtype: float64

Теперь приступим к обучению и исследованию моделей.

Про SMOTE: https://machinelearningmastery.com/smote-oversampling-for-imbalanced-classification/

Дерево решений. 

In [35]:
f_one_best = 0
for min_samples_leaf in range (1,7):
    for min_samples_split in range(2,12):
        for depth in range(2, 12): 
            model_tree = DecisionTreeClassifier(max_depth = depth, random_state=12345, 
                                                min_samples_split = min_samples_split, min_samples_leaf = min_samples_leaf)
            model_tree.fit(features_upsampled, target_upsampled)
            predictions = model_tree.predict(features_valid)
            f_one = f1_score(target_valid,predictions)
            accurasy = accuracy_score(target_valid, predictions)
            if f_one > f_one_best:
                model_tree_best = model_tree
                f_one_best = f_one 
print('F1 = ',f_one_best)  

F1 =  0.5548504079782411


In [36]:
predictions = model_tree_best.predict(features_valid)
accurasy = accuracy_score(target_valid, predictions)
print('accuracy = ', accurasy)
probabilities_tree_b = model_tree_best.predict_proba(features_valid)
print('AUC ROC =', roc_auc_score(target_valid, probabilities_tree_b[:,1]))

accuracy =  0.7545
AUC ROC = 0.8063888233379759


Вывод: для дерева решений f1 осталась примерно такой же, а вот accuracy стала ниже. AUC ROC тоже немного увеличилась

Случайный лес.

In [37]:
f_one_best = 0
for est in range(50, 91, 10):
    for min_samples_split in range(2,12):
        for depth in range(2, 12): 
            model_forest = RandomForestClassifier(max_depth = depth, random_state=12345, 
                                                    min_samples_split = min_samples_split,
                                                     n_estimators = est)
            model_forest.fit(features_upsampled, target_upsampled)
            predictions = model_forest.predict(features_valid)
            f_one = f1_score(target_valid, predictions)
            accurasy = accuracy_score(target_valid, predictions)
            if f_one > f_one_best:
                model_forest_best = model_forest
                f_one_best = f_one 
print('F1 = ',f_one_best)   

F1 =  0.6121076233183856


In [38]:
probabilities_valid = model_forest_best.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc = ', auc_roc)

auc_roc =  0.8529515648159716


Уже гораздо лучше получилось! Без балансов классов было 0,52. А тут 0,61. Посмотрим на accurasy.

In [39]:
predictions = model_forest_best.predict(features_valid)
accurasy = accuracy_score(target_valid, predictions)
print('accuracy = ', accurasy)

accuracy =  0.827


Логистическая регрессия.

In [40]:
model_log_2 = LogisticRegression(random_state=12345)
model_log_2.fit(features_upsampled, target_upsampled)
predictions = model_log_2.predict(features_valid)
f_one = f1_score(target_valid,predictions)
print('F1 = ',f_one)
print('accuracy = ' , accuracy_score(target_valid, predictions))

F1 =  0.47822374039282667
accuracy =  0.6945


Данные лучше гораздо лучше, но все же не дотягивает до 0,59.

In [41]:
probabilities = model_log_2.predict_proba(features_valid) 
print(probabilities)
probabilities_one_valid = probabilities[:, 1]

[[0.19259375 0.80740625]
 [0.90501781 0.09498219]
 [0.41752317 0.58247683]
 ...
 [0.4837729  0.5162271 ]
 [0.85368053 0.14631947]
 [0.84786981 0.15213019]]


In [42]:
fpr, tpr, thresholds = roc_curve(target_valid, probabilities_one_valid) 
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print('auc_roc', auc_roc)

auc_roc 0.7541809914691271


Вывод есть: лучшая модель это Случайный Лес. Самые хорошие показатели. 

## Тестирование модели

И наконец протестируем нашу модель. Будем запускать Случайный лес и просмортим все метрики, что у нас есть. features_test, target_test - данные для тестирования.

In [43]:
predict_test = model_forest_best.predict(features_test)
probabilities_test = model_forest_best.predict_proba(features_test) 

In [44]:
f1_test = f1_score(target_test, predict_test)
auc_roc_test = roc_auc_score(target_test, probabilities_test[:,1])
accuracy_test = accuracy_score(target_test, predict_test)
recall_test = recall_score(target_test, predict_test)
pres_test = precision_score(target_test, predict_test)

In [45]:
print('лучшая модель:')
print('F1 = ', f1_test)
print('auc_roc = ', auc_roc_test)
print('accuracy = ', accuracy_test)
print('recall_test = ', recall_test)
print('pres_test = ', pres_test)

лучшая модель:
F1 =  0.6369710467706012
auc_roc =  0.8689972534239827
accuracy =  0.837
recall_test =  0.7009803921568627
pres_test =  0.5836734693877551


Вывод: модель случайного леса показала хороший результат. Мера F1 0,63 - что даже лучше, чем была на валидационной выборке. Метрика AUC ROC 0,83.

In [46]:
predict_test_tree = model_tree_best.predict(features_test)
probabilities_test = model_tree_best.predict_proba(features_test)
probabilities_test = model_tree_best.predict_proba(features_test) 

In [47]:
f1_test = f1_score(target_test, predict_test_tree)
auc_roc_test = roc_auc_score(target_test, probabilities_test[:,1])
accuracy_test = accuracy_score(target_test, predict_test_tree)
recall_test = recall_score(target_test, predict_test_tree)
pres_test = precision_score(target_test, predict_test_tree)

In [48]:
print('дерево:')
print('F1 = ', f1_test)
print('auc_roc = ', auc_roc_test)
print('accuracy = ', accuracy_test)
print('recall_test = ', recall_test)
print('pres_test = ', pres_test)

дерево:
F1 =  0.5765765765765766
auc_roc =  0.8234093260419746
accuracy =  0.765
recall_test =  0.7843137254901961
pres_test =  0.45584045584045585


Вывод: Лучшая модель получилась случайного леса, но она и медленнее работает.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*